# Modeling CART-mRNA Nanoparticle Volume Fractions

This notebook includes code to model the volume fractions of water, mRNA, and the various components of CART polymers (initiator, lipid block, cationic block) within a CART-mRNA nanoparticle. The calculations implemented here are explained in detail in the Word document called "Updated_mRNA_CART_Volume_Calculations_IsaacA_091820", and the chemical structures of CART polymers referenced are shown as embedded images in that document.


In [67]:
import numpy as np
import scipy.constants as constants

This function creates a numpy array (17 columns, 1 for each piece of information we want to record) for each CART (lipid, cation, block numbers, NP radius). The initiator isn't included since it's the same for all. Then calculate volume fractions of mRNA, lipid, cation, mRNA numbers, and lipid numbers for some range of charge ratios (0.5, 1, 5, 10, 20) and percent water (20, 40, 60, 80). Iterate through all charge ratios and water percentages. Finally, save the completed array as csv file.

In [79]:
from numpy import savetxt
import pandas as pd

# The following high-throughput code works!!

In [64]:
# try V2
charge_ratios = [0.5,0.5,0.5,0.5,1,1,1,1,5,5,5,5,5,10,10,10,10,10,20,20,20,20,20]
percent_water = [20, 40, 60, 80,20, 40, 60, 80,20, 40, 60, 80,20, 40, 60, 80,20, 40, 60, 80]

array = np.zeros((20,7)) # 20 rows, 5 columns(volume fractions that spit out)
# enter inputs for the function
def calc_volume_fraction(cart_lipid, cart_cation, cart_lipid_block_num, cart_cation_block_num, particle_radius):
    #"This code calculates a cart molecule's volume fractions, number of mRNA and carts per nano
   # particle and stores them in an excel file."
    
    # initialize charge ratio and percentage water
    cr = int(0)
    pw = int(0)
    
    cart_lipid = str(cart_lipid)
    cart_cation = str(cart_cation)

    for cr in range(0,20): 
        charge_ratio = charge_ratios[cr]
        water_mass_percent = percent_water[pw]
    
        single_polymer_volume = calculate_cart_volume('Benzyl', cart_lipid, \
                        cart_lipid_block_num,cart_cation,cart_cation_block_num, output='volume')

        single_polymer_mass = calculate_cart_mass('Benzyl', cart_lipid, cart_lipid_block_num, \
                                    cart_cation, cart_cation_block_num, output='mass')

        carts_per_mRNA = calculate_cart_polymers_per_mRNA(charge_ratio,cart_cation,cart_cation_block_num)

        mRNA_molecules,cart_to_mRNA, water_to_mRNA = calculate_mRNA_cart_water_ratio(water_mass_percent, \
                        charge_ratio, cart_cation, cart_cation_block_num, single_polymer_mass)

        molecules_mRNA_per_particle, molecules_polymer_per_particle, molecules_water_per_particle \
                    = calculate_molecules_per_particle(particle_radius, water_mass_percent, charge_ratio, \
                    cart_cation, cart_cation_block_num, single_polymer_mass, single_polymer_volume)
    
        volume_mRNA_per_particle, volume_polymer_per_particle, volume_water_per_particle \
                    = calculate_component_volumes_per_particle(molecules_mRNA_per_particle, \
                    molecules_polymer_per_particle, molecules_water_per_particle, single_polymer_volume)
    
        mRNA_np_fraction, water_np_fraction, initiator_np_fraction, lipid_np_fraction, \
            cation_np_fraction = calculate_particle_volume_fractions(volume_mRNA_per_particle, \
            volume_polymer_per_particle, volume_water_per_particle, 'Benzyl', cart_lipid, \
            cart_lipid_block_num, cart_cation, cart_cation_block_num)
            
        array[cr][0] = mRNA_np_fraction
        array[cr][1] = water_np_fraction
        array[cr][2] = initiator_np_fraction
        array[cr][3] = lipid_np_fraction
        array[cr][4] = cation_np_fraction
        array[cr][5] = molecules_mRNA_per_particle
        array[cr][6] = molecules_polymer_per_particle
           
        cr += 1
        pw += 1
        #pw = int(0)
    
    df=pd.DataFrame(array, columns = ['mRNA_np_fraction', 'water_np_fraction', 'initiator_np_fraction', \
                                      'lipid_np_fraction','cation_np_fraction','mRNA_number','CART_number'], \
                    index = ['0.5_20','0.5_40','0.5_60','0.5_80','1_20','1_40','1_60','1_80','5_20','5_40',\
                             '5_60','5_80','10_20','10_40','10_60','10_80','20_20','20_40','20_60','20_80'])
#    print(df)    
    name = str(cart_lipid)+"_" + str(cart_lipid_block_num) +"_"+ str(cart_cation) +"_"+ str(cart_cation_block_num)+".xlsx"
#    print(name)
    df.to_excel(name)
#    savetxt('name.csv', array, delimiter=',') # not sure??

In [23]:
calc_volume_fraction('Oleyl', 'Glycine', 10, 11, 100)

In [24]:
calc_volume_fraction('Dodecyl', 'Glycine', 13, 11, 100)

In [25]:
calc_volume_fraction('Dodecyl', 'Pip', 14, 8, 100)

In [26]:
calc_volume_fraction('Nonenyl', 'Glycine', 10, 10, 100)

In [27]:
calc_volume_fraction('Dodecyl', 'Lysine', 14, 8, 100)

In [20]:
calc_volume_fraction('Dodecyl', 'Ornithine', 15, 5, 100)

In [ ]:
#calc_volume_fraction('Lip', 'Ornithine', )

# Calculate all integer combinations of blocks

In [65]:
array = np.zeros((256,8))
df = pd.DataFrame(array, columns = ['CART','mRNA_np_fraction', 'water_np_fraction', 'initiator_np_fraction', \
                                      'lipid_np_fraction','cation_np_fraction','mRNA_number','CART_number'])
particle_radius = 100

def diff_block_ratios(cart_lipid, cart_cation, charge_ratio = 10, water_mass_percent = 60):
    # This code calculates volume fractions, number of mRNA, and number of CARTs per nanopartile
    # at different block ratios, storing them in an excel file
    
    cart_lipid_block_num = 5
    row = 0
    for cart_lipid_block_num in range(5,21):
        cart_cation_block_num = 5
        for cart_cation_block_num in range(5,21):
            single_polymer_volume = calculate_cart_volume('Benzyl', cart_lipid,cart_lipid_block_num,cart_cation,cart_cation_block_num, output='volume')
            single_polymer_mass = calculate_cart_mass('Benzyl', cart_lipid, cart_lipid_block_num, cart_cation, cart_cation_block_num, output='mass')
            carts_per_mRNA = calculate_cart_polymers_per_mRNA(charge_ratio,cart_cation,cart_cation_block_num)
            mRNA_molecules,cart_to_mRNA, water_to_mRNA = calculate_mRNA_cart_water_ratio(water_mass_percent, charge_ratio, cart_cation, cart_cation_block_num, single_polymer_mass)

            molecules_mRNA_per_particle, molecules_polymer_per_particle, molecules_water_per_particle \
                    = calculate_molecules_per_particle(particle_radius, water_mass_percent, charge_ratio, \
                    cart_cation, cart_cation_block_num, single_polymer_mass, single_polymer_volume)
    
            volume_mRNA_per_particle, volume_polymer_per_particle, volume_water_per_particle \
                    = calculate_component_volumes_per_particle(molecules_mRNA_per_particle, \
                    molecules_polymer_per_particle, molecules_water_per_particle, single_polymer_volume)
    
            mRNA_np_fraction, water_np_fraction, initiator_np_fraction, lipid_np_fraction, \
                cation_np_fraction = calculate_particle_volume_fractions(volume_mRNA_per_particle, \
                volume_polymer_per_particle, volume_water_per_particle, 'Benzyl', cart_lipid, \
                cart_lipid_block_num, cart_cation, cart_cation_block_num)
            df.iloc[row, 0] = str(cart_lipid) + "_" + str(cart_lipid_block_num) + '_' + str(cart_cation) + '_' + str(cart_cation_block_num)
            df.iloc[row, 1] = mRNA_np_fraction
            df.iloc[row, 2] = water_np_fraction
            df.iloc[row, 3] = initiator_np_fraction
            df.iloc[row, 4] = lipid_np_fraction
            df.iloc[row, 5] = cation_np_fraction
            df.iloc[row, 6] = molecules_mRNA_per_particle
            df.iloc[row, 7] = molecules_polymer_per_particle
            #array[row][6] = molecules_polymer_per_particle
        
            cart_cation_block_num += 1
            row +=1
        
        cart_lipid_block_num += 1    
        
    df.set_index(list(df)[0])
#    print(df)    
    name = str(cart_lipid)+"_"+ str(cart_cation) +".xlsx"
#    print(name)
    df.to_excel(name)

In [78]:
diff_block_ratios('Dodecyl', 'Glycine')

In [79]:
diff_block_ratios('Dodecyl','Lysine')

In [80]:
diff_block_ratios('Nonenyl','Glycine')

In [81]:
diff_block_ratios('Oleyl','Glycine')

In [82]:
diff_block_ratios('Dodecyl','Pip')

In [17]:
diff_block_ratios('Oleyl','Pip')

In [18]:
diff_block_ratios('Oleyl','Lysine')

In [19]:
diff_block_ratios('Nonenyl','Pip')

In [50]:
diff_block_ratios('Lipoate','Glycine')

In [66]:
diff_block_ratios('Dodecyl','Serine')

In [20]:
diff_block_ratios('Nonenyl','Lysine')

### calculate volume fractions for Nonenyl Oleyl, assuming lipid:cation ration is 1:1

In [82]:
array = np.zeros((256,8))
df = pd.DataFrame(array, columns = ['CART','mRNA_np_fraction', 'water_np_fraction', 'initiator_np_fraction', \
                                      'lipid_np_fraction','cation_np_fraction','mRNA_number','CART_number'])
particle_radius = 100

def diff_block_ratios2(cart_cation, cart_lipid  = 'Nonenyl Oleyl', charge_ratio = 10, water_mass_percent = 60):
    # This code calculates volume fractions, number of mRNA, and number of CARTs per nanopartile
    # at different block ratios, storing them in an excel file
    
    cart_lipid_block_num = 5
    row = 0
    for cart_lipid_block_num in range(5,21):
        cart_cation_block_num = 5
        for cart_cation_block_num in range(5,21):
            single_polymer_volume = calculate_cart_volume('Benzyl', cart_lipid,cart_lipid_block_num,cart_cation,cart_cation_block_num, output='volume')
        
            single_polymer_mass = calculate_cart_mass('Benzyl', cart_lipid, cart_lipid_block_num, cart_cation, cart_cation_block_num, output='mass')
            
            carts_per_mRNA = calculate_cart_polymers_per_mRNA(charge_ratio,cart_cation,cart_cation_block_num)
            mRNA_molecules,cart_to_mRNA, water_to_mRNA = calculate_mRNA_cart_water_ratio(water_mass_percent, charge_ratio, cart_cation, cart_cation_block_num, single_polymer_mass)

            molecules_mRNA_per_particle, molecules_polymer_per_particle, molecules_water_per_particle \
                    = calculate_molecules_per_particle(particle_radius, water_mass_percent, charge_ratio, \
                    cart_cation, cart_cation_block_num, single_polymer_mass, single_polymer_volume)
    
            volume_mRNA_per_particle, volume_polymer_per_particle, volume_water_per_particle \
                    = calculate_component_volumes_per_particle(molecules_mRNA_per_particle, \
                    molecules_polymer_per_particle, molecules_water_per_particle, single_polymer_volume)
    
            mRNA_np_fraction, water_np_fraction, initiator_np_fraction, lipid_np_fraction, \
                cation_np_fraction = calculate_particle_volume_fractions(volume_mRNA_per_particle, \
                volume_polymer_per_particle, volume_water_per_particle, 'Benzyl', cart_lipid, \
                cart_lipid_block_num, cart_cation, cart_cation_block_num)
            
            
            df.iloc[row, 0] = str('Nonenyl')+ str(cart_lipid_block_num) + str('Oleyl') + str(cart_lipid_block_num) + '_' + str(cart_cation) + str(cart_cation_block_num)
            df.iloc[row, 1] = mRNA_np_fraction
            df.iloc[row, 2] = water_np_fraction
            df.iloc[row, 3] = initiator_np_fraction
            df.iloc[row, 4] = lipid_np_fraction
            df.iloc[row, 5] = cation_np_fraction
            df.iloc[row, 6] = molecules_mRNA_per_particle
            df.iloc[row, 7] = molecules_polymer_per_particle
        
            cart_cation_block_num += 1
            row +=1
        
        cart_lipid_block_num += 1    
        
    df.set_index(list(df)[0])   
    name = str(cart_lipid)+"_"+ str(cart_cation) +'_'+ str('one to one')+".xlsx"
    df.to_excel(name)

In [83]:
diff_block_ratios2('Ornithine')

### vary all 3 variables (2 lipids and 1 cation)

In [20]:
array = np.zeros((4096,8))
df = pd.DataFrame(array, columns = ['CART','mRNA_np_fraction', 'water_np_fraction', 'initiator_np_fraction', \
                                      'lipid_np_fraction','cation_np_fraction','mRNA_number','CART_number'])
particle_radius = 100

def diff_block_ratios2(cart_lipid1, cart_lipid2, cart_cation, charge_ratio = 10, water_mass_percent = 60):
    # This code calculates volume fractions, number of mRNA, and number of CARTs per nanopartile
    # at different block ratios, storing them in an excel file
    
    cart_lipid1_block_num = 1 
    row = 0
    for cart_lipid1_block_num in range(5,20): # changed to 1 thru 15, anything above 20 would be hard to synthesize
        cart_cation_block_num = 5
        for cart_lipid2_block_num in range(5,20):
            for cart_cation_block_num in range(5,20):
                single_polymer_volume = calculate_cart_volume('Benzyl', cart_lipid,cart_lipid_block_num,cart_cation,cart_cation_block_num, output='volume')
                single_polymer_mass = calculate_cart_mass('Benzyl', cart_lipid, cart_lipid_block_num, cart_cation, cart_cation_block_num, output='mass')
                carts_per_mRNA = calculate_cart_polymers_per_mRNA(charge_ratio,cart_cation,cart_cation_block_num)
                mRNA_molecules,cart_to_mRNA, water_to_mRNA = calculate_mRNA_cart_water_ratio(water_mass_percent, charge_ratio, cart_cation, cart_cation_block_num, single_polymer_mass)

                molecules_mRNA_per_particle, molecules_polymer_per_particle, molecules_water_per_particle \
                    = calculate_molecules_per_particle(particle_radius, water_mass_percent, charge_ratio, \
                    cart_cation, cart_cation_block_num, single_polymer_mass, single_polymer_volume)
    
                volume_mRNA_per_particle, volume_polymer_per_particle, volume_water_per_particle \
                    = calculate_component_volumes_per_particle(molecules_mRNA_per_particle, \
                    molecules_polymer_per_particle, molecules_water_per_particle, single_polymer_volume)
    
                mRNA_np_fraction, water_np_fraction, initiator_np_fraction, lipid_np_fraction, \
                cation_np_fraction = calculate_particle_volume_fractions(volume_mRNA_per_particle, \
                volume_polymer_per_particle, volume_water_per_particle, 'Benzyl', cart_lipid, \
                cart_lipid_block_num, cart_cation, cart_cation_block_num)
                
                df.iloc[row, 0] = str(cart_lipid1) + "_" + str(cart_lipid1_block_num) + \
                            str(cart_lipid2) + "_" + str(cart_lipid2_block_num) + \
                            '_' + str(cart_cation) + '_' + str(cart_cation_block_num)
                df.iloc[row, 1] = mRNA_np_fraction
                df.iloc[row, 2] = water_np_fraction
                df.iloc[row, 3] = initiator_np_fraction
                df.iloc[row, 4] = lipid_np_fraction
                df.iloc[row, 5] = cation_np_fraction
                df.iloc[row, 6] = molecules_mRNA_per_particle
                df.iloc[row, 7] = molecules_polymer_per_particle
            
        
                cart_cation_block_num += 1
                row +=1
        
        cart_lipid_block_num += 1    
        
    df.set_index(list(df)[0])
#    print(df)    
    name = str(cart_lipid1)+"_"+ str(cart_lipid2)+"_"+ str(cart_cation)+"_" +".xlsx"
#    print(name)
    df.to_excel(name)

In [22]:
diff_block_ratios('Dodecyl','Ornithine')

In [21]:
diff_block_ratios2('mixed lipid', 'Ornithine')

In [66]:
interactive_cart_modeling()

CART Initiator: Benzyl
CART Lipid: Dodecyl
CART Lipid Block Number: 5
CART Cation: Glycine
CART Cationic Block Number: 5
Cation to anion charge ratio (i.e. 10:1 = 10): 10
Percent water by mass (i.e. 80% = 80): 60
Cart-mRNA nanoparticle radius (nm) : 100

Volume of single CART polymer (cubic angstroms):
2057.49
Mass of single CART polymer (g):
3.787922087692949e-21
Number of CART polymer molecules per molecule of mRNA:
3858.0
Ratio of molecules of mRNA to CART to water in nanoparticles:
1.0 3858.0 787086.5943396227
Number of molecules of mRNA, CART, and water per nanoparticle:
214.17423438235332 826284.1962471191 168573668.7353026
Volume of mRNA, CART, and water per nanoparticle (cubic angstroms):
101715584.55802035 1700071470.9364848 2387003149.291885
Volume fraction of mRNA, water, CART initiator, CART lipid block, CART cationic block per nanoparticle:
0.0242828071078359 0.5698550255785875 0.018806846707238627 0.29845562269825926 0.08859969790807874
